In [15]:
import pennylane as qml
from pennylane import numpy as numpy

qml.about()



Name: PennyLane
Version: 0.40.0
Summary: PennyLane is a cross-platform Python library for quantum computing, quantum machine learning, and quantum chemistry. Train a quantum computer the same way as a neural network.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /home/ryzzen/.local/lib/python3.12/site-packages
Requires: appdirs, autograd, autoray, cachetools, diastatic-malt, networkx, numpy, packaging, pennylane-lightning, requests, rustworkx, scipy, tomlkit, typing-extensions
Required-by: PennyLane_Lightning

Platform info:           Linux-6.11.0-17-generic-x86_64-with-glibc2.39
Python version:          3.12.3
Numpy version:           1.26.4
Scipy version:           1.11.4
Installed devices:
- default.clifford (PennyLane-0.40.0)
- default.gaussian (PennyLane-0.40.0)
- default.mixed (PennyLane-0.40.0)
- default.qubit (PennyLane-0.40.0)
- default.qutrit (PennyLane-0.40.0)
- default.qutrit.mixed (PennyLane-0.40.0)
- defa

In [16]:
def circuit(params):
    qml.RX(params[0],wires = 0)
    qml.RY(params[1],wires = 0) # 
    return qml.expval(qml.PauliZ(0)) # expectation value is ⟨Z⟩=P(∣0⟩)−P(∣1⟩)

In [21]:
# from braket.devices import Devices
# from braket.aws import AwsDevice

# qpu_device = Devices.Rigetti.AspenM3
# qpu_device_arn = qpu_device.value

# qpu_available = False

# if AwsDevice(qpu_device_arn).is_available:
#     device_arn = qpu_device_arn
#     qpu_available = True

# else:
#     device_arn = "local:pennylane/lightning.qubit"
#     print("AspenM3 is not available. Falling back to lightning.qubit simulator.")

from braket.aws import AwsDevice

# ARN for Rigetti Aspen-M3
qpu_device_arn = "arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3"

# Load the device
qpu_device = AwsDevice(qpu_device_arn)

# Check if the device is available
qpu_available = qpu_device.is_available

print(f"QPU Available: {qpu_available}")



NoRegionError: You must specify a region.

In [22]:
shots_list = [5, 10, 1000]
dev = qml.device("default.qubit", wires=2, shots=shots_list)

@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliX(1)), qml.expval(qml.PauliZ(0))
results = circuit(0.5)
results,results[0]

(((-0.2, 0.6), (0.2, 1.0), (0.018, 0.856)), (-0.2, 0.6))

In [24]:
# def my_first_quantum_function(theta):
import numpy as np
#     qml.RX(theta, wires = 0)
#     qml.PauliY(wires = 1)
#     # qml.Hadamard(wires = 0)
#     qml.Hadamard(wires = 1)

#     return qml.state()
# dev = qml.device("default.qubit", wires = 2)

# import pennylane as qml

# Define a quantum device
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def my_first_quantum_function(theta):
    qml.RX(theta, wires=0) #This applies a rotation gate around the X-axis of the Bloch sphere by an angle theta.
    qml.PauliY(wires=1) #  Pauli-Y operation to qubit 1.
    qml.Hadamard(wires=0) #Hadamard gate creates a superposition from a basis state. 
#     ∣0⟩→ ∣0⟩+∣1⟩ /sqrt(2)
# ​∣1⟩→ ∣0⟩−∣1⟩/sqrt(2) ​

    qml.Hadamard(wires=1) #

    return qml.state()  # Requires `shots=None` for state simulation

# Call the quantum function
theta = 0.5  # Example parameter
quantum_state = my_first_quantum_function(theta)

print(quantum_state) 
print("\n") # Prints the quantum state vector
my_first_qnode = qml.QNode(my_first_quantum_function, dev) 
print(my_first_qnode(np.pi/4))
# The list above represents the amplitudes of the state in the computational basis {|00),|01),|10).|11)}


[ 0.12370198+0.48445621j -0.12370198-0.48445621j -0.12370198+0.48445621j
  0.12370198-0.48445621j]


[ 0.19134172+0.46193977j -0.19134172-0.46193977j -0.19134172+0.46193977j
  0.19134172-0.46193977j]


In [25]:
def subcircuit_1(angle):
    qml.RX(angle,wires = 0)
    qml.PauliY(wires=1)

def subcircuit_2():
    qml.Hadamard(wires = 0)
    qml.CNOT(wires = [0,1])

def full_circuit(theta,phi):
    subcircuit_1(theta)
    subcircuit_2()
    subcircuit_1(phi)

# print(qml.draw(full_circuit(.3,.2)))
theta = 0.3
phi = 0.2 
print(qml.draw(full_circuit)(theta,phi))



0: ──RX(0.30)──H─╭●──RX(0.20)─┤  
1: ──Y───────────╰X──Y────────┤  


In [30]:
dev = qml.device('default.qubit', wires=2)
@qml.qnode(dev)
def circuit(state=None):
    qml.StatePrep(state,wires = range(2)) #prepares a custom quantum state.

    return qml.state()

state = circuit([1/2,1/2,1/2,1/2])   #represents a valid quantum state, where the sum of the squared amplitudes must equal 1. input vector is 1/2{|00),|01),|10).|11)} each state has equal prob no phase difference
print(state)


# Using qml.StatePrep write the state_preparation QNode that prepares a quantum state proportional to

# Note that state_preparation takes the complex numbers
# A(),B(),G() as arguments

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]


In [ ]:
# outcomes of an arbitrary number of quantum measurements, i.e., return a list of samples
# or based on the probabilities given by the input state.


def measure_state(state, num_meas):
    # COMPUTE THE MEASUREMENT OUTCOME PROBABILITIES
    return np.random.choice(2,num_meas, p =[np.abs(state[0])**2, np.abs(state[1])**2])

    # RETURN A LIST OF SAMPLE MEASUREMENT OUTCOMES

    pass

In [ ]:
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)


def apply_u(state):
    """Apply a quantum operation.
    
    Args:
        state (np.array[complex]): A normalized quantum state vector.

    Returns:
        np.array[complex]: The output state after applying U.
    """
    return np.dot(U, state)
def measure_state(state,num_meas):
    p_alpha = np.abs(state[0])**2
    p_beta = np.abs(state[1])**2
    meas_outcome = np.random.choice([0,1],p = [p_alpha,p_beta],size = num_meas)
    return meas_outcome
def quantum_algorithm():
    state = np.array([1,0])
    state1 = apply_u(state)

    return measure_state(state1,100)
    pass
quantum_algorithm()


array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0])

In [11]:
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
def apply_u(state):
    return np.dot(U,state)

# same thing as above but simplier